<a href="https://colab.research.google.com/github/csabiu/Astrostatistics/blob/main/Astro_coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install some packages and import them

In [ ]:
pip install astroquery

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import (SkyCoord, Distance, Galactic,
                                 EarthLocation, AltAz)
import astropy.coordinates as coord
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import AltAz
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file
from astroquery.gaia import Gaia

Define a target object NGC188 - an open star cluster

different ways of defining in astropy

In [ ]:
ngc188_center = SkyCoord(11.798*u.deg, 85.244*u.deg, frame='icrs' )
ngc188_center

In [ ]:
ngc188_center = SkyCoord('00h47m11.5s', '85d14m38s', frame='icrs')
ngc188_center

In [ ]:
ngc188_center = SkyCoord('00:47:11.5 85:14:38', unit=(u.hour, u.deg), frame='icrs')
ngc188_center

In [ ]:
ngc188_center = SkyCoord.from_name('NGC 188')
ngc188_center

Lets search the GAIA database for stars with 0.5 degrees of NGC188 center

In [ ]:
Gaia.ROW_LIMIT = 10000  # Set the row limit for returned data

job = Gaia.cone_search_async(ngc188_center, radius=0.5*u.deg)
ngc188_table = job.get_results()

# only keep stars brighter than G=19 magnitude
ngc188_table = ngc188_table[ngc188_table['phot_g_mean_mag'] < 19*u.mag]

In [ ]:
ngc188_table.colnames

In [ ]:
cols = [
    'source_id',
    'ra',
    'dec',
    'parallax',
    'parallax_error',
    'pmra',
    'pmdec',
    'radial_velocity',
    'phot_g_mean_mag',
    'phot_bp_mean_mag',
    'phot_rp_mean_mag'
]
ngc188_table[cols].write('gaia_results.fits', overwrite=True)



In [ ]:
ngc188_table = QTable.read('gaia_results.fits')
len(ngc188_table)

In [ ]:
ra=ngc188_table['ra']
dec=ngc188_table['dec']
plt.plot(ra,dec,',')
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')

Can you give an estimate of the distance to NGC188 in units of kpc?

In [ ]:
def coordinates_aitoff_plot(coords):
    fig, ax = plt.subplots(figsize=(10, 4),
                           subplot_kw=dict(projection="aitoff"))

    sph = coords.spherical
    cs = ax.scatter(-sph.lon.wrap_at(180*u.deg).radian,
                    sph.lat.radian,
                    c=sph.distance.value)

    def fmt_func(x, pos):
        val = coord.Angle(-x*u.radian).wrap_at(360*u.deg).degree
        return f'${val:.0f}' + r'^{\circ}$'

    ticker = mpl.ticker.FuncFormatter(fmt_func)
    ax.xaxis.set_major_formatter(ticker)

    ax.grid()

    cb = fig.colorbar(cs)
    cb.set_label('distance [pc]')

    return fig, ax



In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/csabiu/astronomy/main/Cantat-Gaudin-open-clusters.ecsv

Load a file of open star clusters

You may need to download from guthub and upload to here manually

In [ ]:
tbl = QTable.read('Cantat-Gaudin-open-clusters.ecsv')
#!head Cantat-Gaudin-open-clusters.ecsv.2
#!rm Cantat-Gaudin-open-cluster*

In [ ]:
open_cluster_c = SkyCoord(
    ra=tbl['ra'],
    dec=tbl['dec'],
    distance=tbl['distance'],
    frame='icrs')
len(open_cluster_c)



In [ ]:
#lets look at the 1st 4 clusters

In [ ]:
open_cluster_c[:4]

In [ ]:
fig, ax = coordinates_aitoff_plot(open_cluster_c)
ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')

lets translate to Galactic coordinates

In [ ]:
open_cluster_gal = open_cluster_c.galactic

In [ ]:
fig, ax = coordinates_aitoff_plot(open_cluster_gal);
ax.set_xlabel('Galactic longitude, $l$ [deg]')
ax.set_ylabel('Galactic latitude, $b$ [deg]')

Can you give an estimate of the distance to these star clusters using GAIA?
Do the distances match?